In [1]:
import requests, bs4
import pandas as pd
from lxml import html #html.lxml 로 써진 것. 즉 html.lxml 로 써야하는 명령어를 lxml만써도 작동하게해줌.
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from pprint import pprint
import numpy as np

In [5]:
# 테스트

# 최종정리코드.
# 파일불러오기
df = pd.read_excel("rawdata/aptexcel202010.xlsx" )
df1 = df.copy()
list = []

#df2.values[14]  column 이름 위치확인
list =  df1.values[14]
list
# column이름 변경 
df1.columns = list
df1.sample()


# 불필요한내용 삭제  (다만 인덱싱을 다시 0부터 시작하게 못하긴했음.)
df1 = df1.drop(df1.index[0:15])
df1.head()


# datetime으로 만들기

df1["년월"] = df1["계약년월"].copy()
#format=%Y%m 으로 포맷 바꿔주기.
df1["년월"] = pd.to_datetime(df1["년월"], format='%Y%m', errors='ignore')

df1


#스트링 숫자로 바꿔주기.
df1["금액"] = pd.to_numeric(df1["거래금액(만원)"],errors ='coerce')
df1["면적"]=pd.to_numeric(df1["전용면적(㎡)"],errors ='coerce')
df1["건축년도"]=pd.to_numeric(df1["건축년도"],errors ='coerce')


#해제사유등 필요없는것 제거하기.

# axis=1을 쓰면 열을 지우고 df.drop([0, 1], axis=0) 처럼 0을쓰면 인덱스(행)을 지움.
# 주의 행의 이름은 B랑 C가 아닌 0 1인데 이는 스트링이 아니므로 ''없이 하면됨.
df2 = df1.drop(["해제사유발생일", "계약년월","단지명","계약일","층","도로명","번지","본번","거래금액(만원)","전용면적(㎡)","부번"]
               ,axis=1).copy()
df2


 #지역으로 나누기.
df2["서울"] = df2["시군구"].str.contains("서울", case=False, na=False).astype(int)
df2["서울"].sum()
df2["대전"] = df2["시군구"].str.contains("대전", case=False, na=False).astype(int)
df2["대전"].sum()

location =[]
location = ["강원도",     
            "경기도"  ,  
            "경상남도" ,    
            "경상북도"  ,   
            "광주광역"     ,
            "대구광역"  ,
            "대전광역"   ,
            "부산광역"   ,
            "서울특별"    ,
            "세종특별",
             "울산광역"    ,
            "인천광역"    ,
            "전라남도" ,    
            "전라북도"  ,   
            "제주특별"    ,
            "충청남도" ,   
            "충청북도"  ] 

for i in location :
    df2[i] =  df2["시군구"].str.contains(i, case=False, na=False).astype(int)
    df2.loc[(df2[i] ==  1) , '지역'] = i


#확인
df2["지역"].value_counts()
df2["강원도"].sum()+df2["경기도"].sum()+df2["경상남도"].sum()+df2["경상북도"].sum()+df2["광주광역"].sum()+df2["대구광역"].sum()+df2["대전광역"].sum()+df2["부산광역"].sum()+df2["서울특별"].sum()+df2["세종특별"].sum()+df2["울산광역"].sum()+df2["인천광역"].sum()+df2["전라남도"].sum()+df2["전라북도"].sum()+df2["제주특별"].sum()+df2["충청남도"].sum() +df2["충청북도"].sum()

    
#크기로 나누기
df2.loc[(df2['면적'] >  60) & (df2['면적'] <= 85), '크기'] = '중형' 
df2.loc[(df2['면적'] <=  60),'크기'] = '소형'
df2.loc[(df2['면적'] > 85), '크기'] = '대형'
#확인 
df2['크기'].value_counts()

df2

,시군구,건축년도,년월,금액,면적,서울,대전,강원도,지역,경기도,...,서울특별,세종특별,울산광역,인천광역,전라남도,전라북도,제주특별,충청남도,충청북도,크기
15,강원도 강릉시 견소동,1997,2020-10-01,17500,84.945,0,0,1,강원도,0,...,0,0,0,0,0,0,0,0,0,중형
16,강원도 강릉시 견소동,1997,2020-10-01,15000,59.800,0,0,1,강원도,0,...,0,0,0,0,0,0,0,0,0,소형
17,강원도 강릉시 견소동,1997,2020-10-01,16800,84.945,0,0,1,강원도,0,...,0,0,0,0,0,0,0,0,0,중형
18,강원도 강릉시 견소동,2005,2020-10-01,20900,84.990,0,0,1,강원도,0,...,0,0,0,0,0,0,0,0,0,중형
19,강원도 강릉시 견소동,2005,2020-10-01,18300,59.990,0,0,1,강원도,0,...,0,0,0,0,0,0,0,0,0,소형
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71966,충청북도 충주시 호암동,2019,2020-10-01,34700,84.960,0,0,0,충청북도,0,...,0,0,0,0,0,0,0,0,1,중형
71967,충청북도 충주시 호암동,2002,2020-10-01,16000,84.680,0,0,0,충청북도,0,...,0,0,0,0,0,0,0,0,1,중형
71968,충청북도 충주시 호암동,2002,2020-10-01,16400,84.680,0,0,0,충청북도,0,...,0,0,0,0,0,0,0,0,1,중형
71969,충청북도 충주시 호암동,2002,2020-10-01,14000,84.680,0,0,0,충청북도,0,...,0,0,0,0,0,0,0,0,1,중형
